In [ ]:
@decorate
def target():
    print("running target()")

# equal to
def target():
    print("running target()")

target = decorate(target)

In [1]:
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print("running target")

target()

running inner()


a key feature of decorators is that they run right after the decorated function is defined. That is usually at import time

In [3]:
registry = []

def register(func):
    print("running register(%s)" % func)
    registry.append(func)
    return func

@register
def f1():
    print("running f1()")

@register
def f2():
    print("running f2()")

def f3():
    print("running f3()")

def main():
    print("running main()")
    print("registry ->", registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x0000014277DE1280>)
running register(<function f2 at 0x0000014277DE1F70>)
running main()
registry -> [<function f1 at 0x0000014277DE1280>, <function f2 at 0x0000014277DE1F70>]
running f1()
running f2()
running f3()


In [ ]:
# variable scope
def f1(a):
    print(a)
    print(b)

f1(3)
# throwing error say b is not defined

In [4]:
b = 6
# def f2(a):
#     print(a)
#     print(b)
#     b = 9
#
# f2(3)

# throwing error local variable `b` referenced before assignment

# in order to fix above do this

def f2(a):
    global b
    print(a)
    print(b)
    b = 9

# print 3 6
f2(3)

# print 6
print(b)

3
6


In [4]:
# avg with class

class Average():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

avg = Average()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [9]:
# avg with higher-order function

def make_average():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

avg = make_average()
print(avg(10))
print(avg(11))
print(avg(12))

# within averager, series is a free variable
# where python keep local variable
print(avg.__code__.co_varnames)

# where python keep freevars
print(avg.__code__.co_freevars)


print(avg.__closure__)

# the actual value where python keep the freevars
print(avg.__closure__[0].cell_contents)

10.0
10.5
11.0
('new_value', 'total')
('series',)
(<cell at 0x00000230D6151A90: list object at 0x00000230D616DF40>,)
Create a new cell object.

  contents
    the contents of the cell. If not specified, the cell will be empty,
    and 
 further attempts to access its cell_contents attribute will
    raise a ValueError.
[10, 11, 12]


In [10]:
# a broken implementation
# def make_averager():
#     count = 0
#     total = 0
#
#     def averager(new_value):
#         count += 1
#         total += new_value
#         return total / count
#     return averager

# this is not working because count += 1
# is actually equal to count = count + 1
# for immutable types like numbers, strings, tpules,
# we can only do read, not update.
# In order to fix that, we can use nonlocal declaration which let us
# flag a variable as a free variable even when it is assigned a
# new value within the function

def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))

10.0
10.5


In [12]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result

    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print("*" * 40, "calling snooze(.123)")
    snooze(.123)
    print("*" * 40, "calling factorial(6)")
    print("6 != ", factorial(6))

**************************************** calling snooze(.123)
[0.12378880s] snooze(0.123) -> None
**************************************** calling factorial(6)
[0.00000060s] factorial(1) -> 1
[0.00001690s] factorial(2) -> 2
[0.00002940s] factorial(3) -> 6
[0.00004100s] factorial(4) -> 24
[0.00005230s] factorial(5) -> 120
[0.00006490s] factorial(6) -> 720
6 !=  720


In [ ]:
# improved version of clock decorator

import time, functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(", ".join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(", ".join(pairs))
        arg_str = ", ".join(arg_lst)
        print("[%0.8fs] %s(%s) -> %r " % (elapsed, name, arg_str, result))
        return result
    return clocked

In [ ]:
# stacked decorators
# @d1
# @d2
# def f():
#     print('f')

# equals to f = d1(d2(f))

In [16]:
# parameterized decorator

registry = set()

def register(active=True):
    def decorate(func):
        print("running register(active=%s)->decorate(%s)" % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@registry(active=False)
def f1():
    print("running f1()")

@registry()
def f2():
    print("running f2()")

def f3():
    print("running f3()")

TypeError: 'set' object is not callable